In [41]:
import cellpose
from macrohet import tools, dataio
import glob
import os
import zarr
import numpy as np
import napari
from tqdm.auto import tqdm

In [13]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

def segment(img, ):
    masks, flows, styles, diams = model.eval(img, diameter=350, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0
Thu Feb 15 16:03:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:65:00.0  On |                  Off |
| 30%   47C    P8              36W / 30

In [4]:
expt_ID = 'ND0002'
location =  'SYNO' #'DATA' # 'NEMO' #
base_dir = f'/mnt/{location}/macrohet_{location.lower()}/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_fn, assay_layout=True,)

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


In [6]:
assay_layout

Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

In [7]:
acq_ID = (3, 4)

In [37]:
zarr_group = zarr.open(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/zarr/{acq_ID}.zarr', mode='r')

In [38]:
images = zarr_group.images

In [39]:
images.shape

(150, 2, 3, 6048, 6048)

In [28]:
img = np.max(images[0,1,:,...], axis = 0)

In [29]:
img.shape

(6048, 6048)

In [30]:
viewer = napari.Viewer()

viewer.add_image(img)

<Image layer 'img' at 0x7f8c65445f40>

In [34]:

model = models.Cellpose(gpu=True, model_type='cyto3')


masks, flows, styles, diams = model.eval(img,
                                         diameter = 350)
                                         # diameter=None, #350
                                         # channels=[0,0],
                                         # flow_threshold=None, 
                                         # cellprob_threshold=0)

In [35]:
viewer.add_labels(masks)

<Labels layer 'masks [1]' at 0x7f8c648c4e20>

In [36]:
from macrohet import notify
notify.send_sys_message()

# Try whole stack overnight

In [ ]:
image_stack = np.max(images[:,1,:,...], axis = 1)

In [ ]:
mask_stack = []
for img in tqdm(image_stack):
    masks, flows, styles, diams = model.eval(img,
                                             diameter = 350)
                                             # diameter=None, #350
                                             # channels=[0,0],
                                             # flow_threshold=None, 
                                             # cellprob_threshold=0)
    mask_stack.append(masks)
segmentation = np.stack(mask_stack, axis = 0)